In [1]:
import os
import time

import anndata as ad
import pandas as pd
import numpy as np

from SpatialQuery import spatial_query_multiple

In [2]:
pd.set_option('display.max_colwidth', 1000)
data_path = "/Users/sa3520/BWH/spatial query/python/data/CZI_kidney"
data_files = os.listdir(data_path)
adatas = [ad.read_h5ad(os.path.join(data_path, data)) for data in data_files]


In [3]:
spatial_key = 'X_spatial'
label_key = 'cell_type'
disease_key = 'disease'

disease_list = [adata.obs[disease_key].unique()[0] for adata in adatas]
disease_list = list(set(disease_list))
print(disease_list)

disease_normal_adatas = [adata for adata in adatas if adata.obs[disease_key].unique()[0] == 'normal']
disease_diabetic_adatas = [adata for adata in adatas if adata.obs[disease_key].unique()[0] == 'diabetic kidney disease']

datasets = ['normal'] * len(disease_normal_adatas) + ['diabetic kidney disease'] * len(disease_diabetic_adatas)

del adatas

cell_types = [adata.obs[label_key] for adata in disease_normal_adatas + disease_diabetic_adatas]
cell_types = pd.concat(cell_types)
# cell_types.value_counts()


# Build data: 0.33s for ~1.5M cells
start_time = time.time()
multi_sp = spatial_query_multiple.spatial_query_multiple(adatas=disease_normal_adatas + disease_diabetic_adatas,
                                                         datasets=datasets,
                                                         spatial_key=spatial_key,
                                                         label_key=label_key,
                                                         )
end_time = time.time()
print(f"time of initializing data and building kd tree is {end_time - start_time} seconds.")

n_normal = [adata.n_obs for adata in disease_normal_adatas]
n_dkd = [adata.n_obs for adata in disease_diabetic_adatas]
print(f"total number of normal data: {np.sum(n_normal)}")
print(f"total number of dkd data: {np.sum(n_dkd)}")

del disease_normal_adatas
del disease_diabetic_adatas


['diabetic kidney disease', 'normal', 'autosomal dominant polycystic kidney disease']
time of initializing data and building kd tree is 0.3346083164215088 seconds.
total number of normal data: 886263
total number of dkd data: 613317


In [4]:
import ipywidgets as widgets
from IPython.display import display

# Function to display the current values of parameters
def display_parameters(k, radius, min_support, is_duplicate):
    selected_datasets = ', '.join([checkbox.description for checkbox in dataset_checkboxes if checkbox.value])
    print(f"Current value of k: {k}")
    print(f"Current value of radius: {radius}")
    print(f"Current value of min_support: {min_support:.2f}")
    print(f"Is duplicate considered: {'Yes' if is_duplicate else 'No'}")
    print(f"Selected datasets: {selected_datasets}")

# Sliders and checkboxes for parameters
k_slider = widgets.IntSlider(value=30, min=1, max=500, step=1, description='k (Number of Neighbors):', continuous_update=False)
min_support_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='Min Support:', continuous_update=False)
radius_slider = widgets.FloatSlider(value=100, min=1, max=500, step=1, description='Radius:', continuous_update=False)
is_duplicate_checkbox = widgets.Checkbox(value=False, description='Consider Duplicates:', disabled=False)

# Create checkboxes for each dataset
datasets = ['All', 'diabetic kidney disease', 'normal']
dataset_checkboxes = [widgets.Checkbox(value=False, description=dataset) for dataset in datasets]

# Handling the "All" checkbox for datasets
def handle_all_datasets_checkbox_change(change):
    if change['new']:  # If "All" is set to True
        for checkbox in dataset_checkboxes[1:]:
            checkbox.value = True
    elif all(checkbox.value for checkbox in dataset_checkboxes[1:]):  # Only uncheck others if all are checked
        for checkbox in dataset_checkboxes[1:]:
            checkbox.value = False

dataset_checkboxes[0].observe(handle_all_datasets_checkbox_change, 'value')

# Group dataset checkboxes in a container
dataset_checkbox_container = widgets.VBox(dataset_checkboxes)

# Interactive display setup without datasets since they are handled separately
parameter_controls = widgets.interactive(display_parameters, k=k_slider, radius=radius_slider, min_support=min_support_slider, is_duplicate=is_duplicate_checkbox)

# Display everything
display(parameter_controls, dataset_checkbox_container)


interactive(children=(IntSlider(value=30, continuous_update=False, description='k (Number of Neighbors):', max…

In [8]:
k = k_slider.value
min_support = min_support_slider.value
is_duplicate = is_duplicate_checkbox.value
radius = radius_slider.value
datasets = [checkbox.description for checkbox in dataset_checkboxes if checkbox.value]

print(k)
print(min_support)
print(is_duplicate)
print(radius)
print(datasets)

30
0.5
False
100.0
['diabetic kidney disease', 'normal']


In [9]:
ct = 'kidney interstitial fibroblast' 
fp_knn = multi_sp.find_fp_knn(
    ct=ct, 
    dataset=datasets,
    k=k,
    min_support=min_support
    
)
fp_knn

,itemsets,support
0,"[endothelial cell, kidney loop of Henle thick ascending limb epithelial cell, leukocyte]",0.645191
1,"[endothelial cell, kidney proximal convoluted tubule epithelial cell, leukocyte]",0.599383
2,"[endothelial cell, leukocyte, macrophage]",0.507830
3,"[endothelial cell, kidney interstitial fibroblast]",0.502528


In [ ]:
motifs = fp_knn['itemsets'].tolist()
motif_enrich_knn = pd.DataFrame()
for motif in motifs:
    tt = multi_sp.motif_enrichment_knn(
        ct=ct, 
        motifs=motif,
        dataset=datasets,
        k=k,
        min_support=min_support,
        dis_duplicates=is_duplicate,
        max_dist=200
    )
    motif_enrich_knn = pd.concat([motif_enrich_knn, tt], ignore_index=True)
motif_enrich_knn

In [ ]:
ct = 'kidney interstitial fibroblast' 
min_support=0.3
d1, d2 = multi_sp.differential_analysis_knn(
    ct=ct, 
    datasets=datasets,
    k=k,
    min_support=min_support
)


In [ ]:
d1[['p-values', 'corrected p-values']]

In [ ]:
d2[['p-values', 'corrected p-values']]

In [ ]:


# ct='leukocyte' # ~200k cells
# ct = 'mesangial cell' # 1.3k cells: dis_duplicates=False: 0.02; dis_duplicates=True: 0.38s
# ct = 'kidney interstitial fibroblast' # ~16k cells, dis_duplicates=False: 0.09s; dis_duplicates=True: 0.36s
# ct = 'kidney loop of Henle thick ascending limb epithelial cell' # ~170k cells, 
# dis_duplicates=False: 0.95s; dis_duplicates=True: 10.24s
# ct = 'endothelial cell' # ~400k cells: dis_duplicates=False: 2.02s; dis_duplicates=True: 9.78s
ct = "kidney proximal convoluted tubule epithelial cell" # ~532k cells: 
# dis_duplicates=False: 2.77s; dis_duplicates=True: 75.98s (69s for frequent patterns, too much patterns)

n_ct = [l==ct for label in multi_sp.labels for l in label]
print(f"number of {ct} in total: {np.sum(n_ct)}")

start_time = time.time()
fp_knn = multi_sp.find_fp_knn(
        ct=ct,
        # dataset='normal',
        # dataset='diabetic kidney disease',
        # dis_duplicates=False,
        dis_duplicates=True,
        min_support=0.5
        )
end_time = time.time()
print(f"{end_time-start_time} seconds for find_fp_knn")

start_time = time.time()
fp_knn_dis_false = multi_sp.find_fp_knn(
        ct=ct,
        dis_duplicates=False,
        min_support=0.5
        )
end_time = time.time()
print(f"{end_time-start_time} seconds for find_fp_knn")



# ct='leukocyte' # ~200k cells
# ct = 'mesangial cell' # 1.3k cells: dis_duplicates=False: 4.68s; dis_duplicates=True: 4.90s
# ct = 'kidney interstitial fibroblast' # ~16k cells, dis_duplicates=False: 4.68s; dis_duplicates=True: 4.94s
# ct = 'kidney loop of Henle thick ascending limb epithelial cell' # ~170k cells, 
# dis_duplicates=False: 4.95s; dis_duplicates=True: 5.25s
# ct = 'endothelial cell' # ~400k cells: dis_duplicates=False: 5.25s; dis_duplicates=True: 5.58s
ct = "kidney proximal convoluted tubule epithelial cell" # ~532k cells: 
# dis_duplicates=False: 5.40s; dis_duplicates=True: 5.81s (69s for frequent patterns, too much patterns)

n_ct = [l==ct for label in multi_sp.labels for l in label]
print(f"number of {ct} is {np.sum(n_ct)}")
motifs = fp_knn['itemsets'][0]
start_time = time.time()
dataset = 'normal'
dataset = 'diabetic kidney disease'
dataset = None
tt = multi_sp.motif_enrichment_knn(
        ct=ct, 
        dataset=dataset, 
        motifs=motifs, 
        k=30, min_support=0.5, dis_duplicates=True
        )
end_time = time.time()
print(f"{end_time - start_time} seconds for motif_enrichment_knn using {dataset} dataset")


motifs = fp_knn_dis_false['itemsets'][0]
start_time = time.time()
# dataset = 'normal'
# dataset = 'diabetic kidney disease'
dataset = None
tt = multi_sp.motif_enrichment_knn(
        ct=ct, 
        dataset=dataset, 
        motifs=motifs, 
        k=30, min_support=0.5, dis_duplicates=False
        )
end_time = time.time()
print(f"{end_time - start_time} seconds for motif_enrichment_knn using {dataset} dataset")



# ct='leukocyte' # ~200k cells
# ct = 'mesangial cell' # 1.3k cells: dis_duplicates=False: 0.02; dis_duplicates=True: 5.08s 
# ct = 'kidney interstitial fibroblast' # ~16k cells, dis_duplicates=False: 0.13s; dis_duplicates=True: 0.46s
# ct = 'kidney loop of Henle thick ascending limb epithelial cell' # ~170k cells, 
# dis_duplicates=False: 1.27s; dis_duplicates=True: 24s (22s for identifying frequent patterns)
ct = 'endothelial cell' # ~400k cells: dis_duplicates=False: 2.69s; dis_duplicates=True: 12.41s (9.4s for frequent patterns)
# ct = "kidney proximal convoluted tubule epithelial cell" # ~532k cells: 
# dis_duplicates=False: 4.06s; dis_duplicates=True: more than 10mins, shut down manually

n_ct = [l==ct for label in multi_sp.labels for l in label]
print(f"number of {ct} in total: {np.sum(n_ct)}")

start_time = time.time()
fp_dist = multi_sp.find_fp_dist(
    ct=ct,
    # dataset=['normal'],
    max_dist=100,
    min_size=0,
    dis_duplicates=True,
    # dis_duplicates=True,
    min_support=0.6)
end_time = time.time()
print(f"{end_time-start_time} seconds for find_fp_dist")

start_time = time.time()
fp_dist_dis_false = multi_sp.find_fp_dist(
        ct=ct,
        max_dist=100,
        min_size=0,
        dis_duplicates=False,
        min_support=0.6
        )
end_time = time.time()
print(f"{end_time-start_time} seconds for find_fp_dist")



# ct='leukocyte' # ~200k cells
# ct = 'mesangial cell' # 1.3k cells: dis_duplicates=False: 5,90s; dis_duplicates=True: 6.49s
# ct = 'kidney interstitial fibroblast' # ~16k cells, dis_duplicates=False: 5.90s; dis_duplicates=True: 6.19s
# ct = 'kidney loop of Henle thick ascending limb epithelial cell' # ~170k cells, 
# dis_duplicates=False: 6.55s; dis_duplicates=True: 6.21s
# ct = 'endothelial cell' # ~400k cells: dis_duplicates=False: 6.97s; dis_duplicates=True: 6.61s
# ct = "kidney proximal convoluted tubule epithelial cell" # ~532k cells: 
# dis_duplicates=False: 7.03s; dis_duplicates=True: 7.33s (69s for frequent patterns, too much patterns)

n_ct = [l==ct for label in multi_sp.labels for l in label]
print(f"number of {ct} is {np.sum(n_ct)}")
motifs = fp_dist['itemsets'][0]
start_time = time.time()
dataset = None
tt = multi_sp.motif_enrichment_dist(
        ct=ct, 
        dataset=dataset, 
        motifs=motifs, 
        max_dist=100.0, min_size=0, min_support=0.6, dis_duplicates=True
        )
end_time = time.time()
print(f"{end_time - start_time} seconds for motif_enrichment_knn using {dataset} dataset")


motifs = fp_dist_dis_false['itemsets'][0]
start_time = time.time()
# dataset = 'normal'
# dataset = 'diabetic kidney disease'
dataset = None
tt = multi_sp.motif_enrichment_dist(
        ct=ct, 
        dataset=dataset, 
        motifs=motifs, 
        max_dist=100.0, min_size=0, min_support=0.6, dis_duplicates=False
        )
end_time = time.time()
print(f"{end_time - start_time} seconds for motif_enrichment_knn using {dataset} dataset")



# ct = 'mesangial cell' # 1.3k cells: 0.05s
# ct = 'kidney interstitial fibroblast' # ~16k cells, 0.13s
# ct = 'kidney loop of Henle thick ascending limb epithelial cell' # ~170k cells, 0.94s
# ct = 'endothelial cell' # ~400k cells: 1.94s
ct = "kidney proximal convoluted tubule epithelial cell" # ~532k cells: 2.61s

start_time = time.time()
fp0, fp1 = multi_sp.differential_analysis_knn(
    ct=ct,
    datasets=['normal', 'diabetic kidney disease'],
    k=30,
    min_support=0.5,
)
end_time = time.time()
print(f"time of running differential_analysis_dist is {end_time - start_time} seconds.")


# ct = 'mesangial cell' # 1.3k cells: 0.08s
# ct = 'kidney interstitial fibroblast' # ~16k cells, 0.21s
# ct = 'kidney loop of Henle thick ascending limb epithelial cell' # ~170k cells, 1.9s
# ct = 'endothelial cell' # ~400k cells: 4.07s
ct = "kidney proximal convoluted tubule epithelial cell" # ~532k cells: 5.66s

start_time = time.time()
fp0, fp1 = multi_sp.differential_analysis_dist(
    ct=ct,
    datasets=['normal', 'diabetic kidney disease'],
    max_dist=100.0,
    min_support=0.5,
)
end_time = time.time()
print(f"time of running differential_analysis_dist is {end_time - start_time} seconds.")




